In [13]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as po
import requests as rq

In [2]:
MAPBOX_TOKEN = 'pk.eyJ1IjoiZ3JvbDIwMjAiLCJhIjoiY2s4bnVxeDk0MTI5MDNqbzJ3N212d3JvNSJ9.g2Pe3QhqbcfGOedTchNgCw'

In [3]:
#read in latest info from https://github.com/datasets/covid-19
df = pd.read_csv("https://raw.githubusercontent.com/datasets/covid-19/master/data/time-series-19-covid-combined.csv")
df.Date = pd.to_datetime(df.Date,format="%Y-%m-%d")
#create population column for later
df["Population"] = np.NaN
df.head()

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population
0,2020-01-22,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN
1,2020-01-23,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN
2,2020-01-24,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN
3,2020-01-25,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN
4,2020-01-26,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,NaN


In [7]:
cinfo = rq.get("https://raw.githubusercontent.com/gregory1506/DashPlotly/master/test.json").json()
df2 = df.copy()
df2["Region"] = np.nan
df2["Sub-Region"] = np.nan
df2["ISO3"] = np.nan
countries = df["Country/Region"].unique()
for country in countries:
    if country in cinfo:
        df2.loc[df2["Country/Region"] == country,["Population","Region","Sub-Region","ISO3"]] = cinfo[country][:]
df2.head(5)

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population,Region,Sub-Region,ISO3
0,2020-01-22,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG
1,2020-01-23,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG
2,2020-01-24,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG
3,2020-01-25,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG
4,2020-01-26,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0,27657145.0,Asia,Southern_Asia,AFG


In [18]:
COUNTRIES = df2["Country/Region"].unique()
DATES = sorted(df2.Date.unique())

In [76]:
def mapbox_center(country=None):
    df_tmp = df2[df2["Country/Region"].isin(country)]
    lat = round(df_tmp.Lat.mean(),3)
    lon = round(df_tmp.Long.mean(),3)
    return dict(lat=lat,lon=lon)

def mapbox_zoom(country=None):
    if set(country) == set(COUNTRIES):
        return 1.07
    else:
        return 3.0

In [77]:
def make_covid_map(case=None,country=None,date=None):
        if country is None or country == []:
            country = COUNTRIES
        elif type(country) == str:
            country = list((country,))
        elif type(country) == list:
            pass
        else:
            print("Only lists or single string accepted for country")
        if date is None:
            date = DATES[-1]
        else:
            date = np.datetime64(date)
        df_tmp = df2[(df2.Date == date) & (df2.Confirmed > 0) & (df2["Country/Region"].isin(country))]
        return {"data" : [go.Scattermapbox(
                                lat=df_tmp.Lat,
                                lon=df_tmp.Long,
                                mode="markers",
                                marker=go.scattermapbox.Marker(
                                        size=df_tmp.Confirmed ** (1/3),
                                        color="red",
                                        opacity=0.7
                                ),
                                text=df_tmp["Country/Region"] + " " + "Confirmed : " + df_tmp["Confirmed"].astype(str),
                                hoverinfo="text"
                                )
                        ],
                "layout" : go.Layout(
                                hovermode="closest",
                                autosize=True,
                                mapbox={"accesstoken":MAPBOX_TOKEN,
                                        "center":mapbox_center(country),
                                        "zoom":mapbox_zoom(country)
                                        },
                                )
        }
fig = go.Figure(make_covid_map(country=["Barbados","US"],date="2020-03-01"))
po.plot(fig,filename="mapbox2.html")

'mapbox2.html'

In [79]:
df2[(df2["Country/Region"] == "China") & (df2.Date == DATES[-1])]

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Population,Region,Sub-Region,ISO3
4987,2020-04-16,China,Anhui,31.8257,117.2264,991.0,984.0,6.0,1.377422e+09,Asia,Eastern_Asia,CHN
5073,2020-04-16,China,Beijing,40.1824,116.4142,593.0,503.0,8.0,1.377422e+09,Asia,Eastern_Asia,CHN
5159,2020-04-16,China,Chongqing,30.0572,107.8740,579.0,570.0,6.0,1.377422e+09,Asia,Eastern_Asia,CHN
5245,2020-04-16,China,Fujian,26.0789,117.9874,353.0,333.0,1.0,1.377422e+09,Asia,Eastern_Asia,CHN
5331,2020-04-16,China,Gansu,37.8099,101.0583,139.0,137.0,2.0,1.377422e+09,Asia,Eastern_Asia,CHN
5417,2020-04-16,China,Guangdong,23.3417,113.4244,1571.0,1471.0,8.0,1.377422e+09,Asia,Eastern_Asia,CHN
5503,2020-04-16,China,Guangxi,23.8298,108.7881,254.0,252.0,2.0,1.377422e+09,Asia,Eastern_Asia,CHN
5589,2020-04-16,China,Guizhou,26.8154,106.8748,146.0,144.0,2.0,1.377422e+09,Asia,Eastern_Asia,CHN
5675,2020-04-16,China,Hainan,19.1959,109.7453,168.0,162.0,6.0,1.377422e+09,Asia,Eastern_Asia,CHN
5761,2020-04-16,China,Hebei,39.5490,116.1306,328.0,315.0,6.0,1.377422e+09,Asia,Eastern_Asia,CHN


In [57]:
mapbox_center(["US","Russia"])

{'lat': 48.545, 'lon': -2.856}

In [89]:
date = DATES[-1]
country = ["US","Canada"]
tmp = df2[(df2.Date == date) & (df2.Confirmed > 0) & (df2["Country/Region"].isin(country))]
tmp1 = tmp.groupby(["Date"]).agg(sum).copy()
# tmp1["Confirmed"][0]
tmp1

,Lat,Long,Confirmed,Recovered,Deaths,Population
Date,,,,,,
2020-04-16,708.8509,-1333.3418,698610.0,54703.0,34174.0,793968331.0


In [96]:
date = DATES[-1]
country = ["US","Canada"]
tmp = df2[(df2.Date.isin(DATES[:DATES.index(date)])) & (df2.Confirmed > 0) & (df2["Country/Region"].isin(country))]
tmp = tmp.groupby(["Date"]).agg(sum).copy()
fig = go.Figure()
fig.add_trace(go.Scatter(x=tmp.index,y=tmp.Deaths,fill='tozeroy',fillcolor="red",mode='none',name="Deaths"))
fig.add_trace(go.Scatter(x=tmp.index,y=tmp.Recovered,fill='tonexty',fillcolor="green",mode='none',name="Recovered"))
fig.add_trace(go.Scatter(x=tmp.index,y=tmp.Confirmed,fill='tonexty',fillcolor="blue",mode='none',name="Confirmed"))
fig.update_layout(showlegend=True,xaxis=dict(title="Progression of Outbreak (Date)"),yaxis=dict(title="Number of cases"))
po.plot(fig,filename="lineplot2.html")

'lineplot2.html'

In [92]:
datf = np.datetime64("2020-04-01")
DATES.index(datf)

70